# 使用 PsychoPy 实现随机点动实验

## 界面

<img src="images/interface_1.png" alt="界面1" width="50%">

<img src="images/interface_2.png" alt="界面2" width="50%">

## 实验结果

<img src="images/results.png" alt="实验结果" width="50%">

In [ ]:
from psychopy import visual, core, event  # type: ignore
import random
import csv

n_trials = 5  # 实验次数
dot_density = 100  # 点云数量
coherence_level = 0.9  # 一致性
motion_duration = 5.0  # 每次点云运动的持续时间（秒）
response_keys = {"left": "left", "right": "right"}  # 按键映射

output_file = "random_dot_motion_experiment.csv"

# 创建窗口
win = visual.Window(size=(1600, 1000), color=(0, 0, 0), units="height")

# 创建随机点云
dot_stim = visual.DotStim(
    win=win,
    nDots=dot_density,
    coherence=coherence_level,
    dotSize=5,
    speed=0.01,
    fieldSize=(0.5, 0.5),
    fieldShape="circle",
    signalDots="same",  # 一致运动的点群在各帧保持一致
    noiseDots="position",  # 噪声点随机改变位置
)

# 提示文本
instruction = visual.TextStim(
    win,
    text="观察点云的主要运动方向：\n左: 按 '←'\n右: 按 '→'\n按空格键开始实验",
    color=(1, 1, 1),
    pos=(0, 0),
    height=0.05,
)

# 实验说明
instruction.draw()
win.flip()
event.waitKeys(keyList=["space"])

results = []

for trial in range(n_trials):
    # 随机选择主要运动方向
    # 左: 180°, 右: 0°
    motion_direction = random.choice([180, 0])
    dot_stim.dir = motion_direction

    trial_clock = core.Clock()  # 记录反应时间
    dot_stim.setAutoDraw(True)

    # 持续刷新屏幕，更新点的位置
    response = None
    rt = None
    is_correct = False
    while trial_clock.getTime() < motion_duration:
        dot_stim.draw()
        win.flip()

        keys = event.getKeys(keyList=response_keys.keys(), timeStamped=trial_clock)
        # 记录首次按键
        if keys and not response:
            response, rt = keys[0]
            is_correct = response_keys[response] == (
                "left" if motion_direction == 180 else "right"
            )
            break

    # 停止
    dot_stim.setAutoDraw(False)
    win.flip()

    results.append(
        {
            "Trial": trial + 1,
            "MotionDirection": "left" if motion_direction == 180 else "right",
            "Response": response,
            "Correct": is_correct,
            "ReactionTime": rt,
        }
    )
    core.wait(0.5)


with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=results[0].keys())
    writer.writeheader()
    writer.writerows(results)

end_text = visual.TextStim(
    win, text="实验结束，谢谢参与！", color=(1, 1, 1), pos=(0, 0), height=0.05
)
end_text.draw()
win.flip()
core.wait(2.0)

win.close()

print(f"实验结果已保存到 {output_file}")